In [ ]:
"""
This code evaluates the L1 loss of our model and the ResNet base model with a dataset has evenly distributed pulse FWHMs from 25fs ~ 70fs
"""
####### Predict the FWHM of arbitrary-duration pulses with MultiRes 
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision.transforms as transforms
import csv
import math
import torch.nn as nn
import torch.nn.functional as F
import itertools
import random
import torchvision
from torchvision.transforms import Compose, CenterCrop, ToTensor, Resize, Normalize
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#---checkpoint
def save_checkpoint(model, epoch, checkpoint_dir, stats):
    """
    Save model checkpoint.
    """
    state = {
        'epoch': epoch,
        'state_dict': model.state_dict(),
        'stats': stats,
    }

    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    filename = os.path.join(checkpoint_dir,
                            'epoch={}.checkpoint.pth.tar'.format(epoch))
    torch.save(state, filename)


# =============================================================================
def restore_checkpoint(model, checkpoint_dir, cuda=False, force=False, pretrain=False):
    """
    If a checkpoint exists, restores the PyTorch model from the checkpoint.
    Returns the model, the current epoch, and training losses.
    """
    def get_epoch(cp):
        return int(cp.split('epoch=')[-1].split('.checkpoint.pth.tar')[0])

    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    cp_files = [file_ for file_ in os.listdir(checkpoint_dir)
                if file_.startswith('epoch=') and file_.endswith('.checkpoint.pth.tar')]
    cp_files.sort(key=lambda x: get_epoch(x))

    if not cp_files:
        print('No saved model parameters found')
        if force:
            raise Exception('Checkpoint not found')
        else:
            return model, 0, []

    # Find latest epoch
    epochs = [get_epoch(cp) for cp in cp_files]

    if not force:
        epochs = [0] + epochs
        print('Which epoch to load from? Choose from epochs below:')
        print(epochs)
        print('Enter 0 to train from scratch.')
        print(">> ", end='')
        inp_epoch = int(input())
        if inp_epoch not in epochs:
            raise Exception("Invalid epoch number")
        if inp_epoch == 0:
            print("Checkpoint not loaded")
            clear_checkpoint(checkpoint_dir)
            return model, 0, []
    else:
        print('Which epoch to load from? Choose from epochs below:')
        print(epochs)
        print(">> ", end='')
        inp_epoch = int(input())
        if inp_epoch not in epochs:
            raise Exception("Invalid epoch number")

    filename = os.path.join(checkpoint_dir, 'epoch={}.checkpoint.pth.tar'.format(inp_epoch))

    print("Loading from checkpoint {}".format(filename))

    if cuda:
        checkpoint = torch.load(filename)
    else:
        # Load GPU model on CPU
        checkpoint = torch.load(filename, map_location=lambda storage, loc: storage)

    try:
        stats = checkpoint['stats']
        if pretrain:
            model.load_state_dict(checkpoint['state_dict'], strict=False)
        else:
            model.load_state_dict(checkpoint['state_dict'])
        print("=> Successfully restored checkpoint (trained for {} epochs)".format(checkpoint['epoch']))
    except:
        print("=> Checkpoint not successfully restored")
        raise

    return model, inp_epoch, stats
# =============================================================================


def clear_checkpoint(checkpoint_dir):
    """
    Delete all checkpoints in directory.
    """
    filelist = [f for f in os.listdir(checkpoint_dir) if f.endswith(".pth.tar")]
    for f in filelist:
        os.remove(os.path.join(checkpoint_dir, f))

    print("Checkpoint successfully removed")

#----plotter
class Plotter:
    def __init__(self, stats=[], name='CNN'):
        self.stats = stats
        self.name = name
        self.axes = self.make_cnn_training_plot()

    def make_cnn_training_plot(self):
        """
        Runs the setup for an interactive matplotlib graph that logs the loss and accuracy
        """
        print('Setting up interactive graph...')
        plt.ion()
        self.fig, axes = plt.subplots(1, 2, figsize=(10, 5))
        plt.suptitle(self.name + ' Training')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Accuracy')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Mean Loss')
        return axes

    def log_cnn_training(self, epoch, csv_location='/content/drive/My Drive/MachineLearningProject2020/CSV/multires_data.csv'):
        """
        Logs the validation accuracy and loss to the terminal - Not currently doing the CSV logging
        """
        valid_acc, valid_loss, train_acc, train_loss = self.stats[-1]
        print('Epoch {}'.format(epoch))
        print('\tValidation Loss: {}'.format(valid_loss))
        print('\tValidation Accuracy: {}'.format(valid_acc))
        print('\tTrain Loss: {}'.format(train_loss))
        print('\tTrain Accuracy: {}'.format(train_acc))
        
    def update_cnn_training_plot(self, epoch):
        """
        Updates the training plot with a new data point for loss and accuracy
        """
        xrange = range(epoch - len(self.stats) + 1, epoch)
        self.axes[0].plot(xrange, [s[0] for s in self.stats[1:]], linestyle='--', marker='o', color='b')
        self.axes[0].plot(xrange, [s[2] for s in self.stats[1:]], linestyle='--', marker='o', color='r')
        self.axes[1].plot(xrange, [s[1] for s in self.stats[1:]], linestyle='--', marker='o', color='b')
        self.axes[1].plot(xrange, [s[3] for s in self.stats[1:]], linestyle='--', marker='o', color='r')
        self.axes[0].legend(['Validation', 'Train'])
        self.axes[1].legend(['Validation', 'Train'])
        plt.pause(0.00001)

    def save_cnn_training_plot(self):
        """
        Saves the training plot to a file
        """
        self.fig.savefig(self.name + '_training_plot.png', dpi=200)

    def hold_training_plot(self):
        """
        Keep the program alive to display the training plot
        """
        plt.ioff()
        plt.show()

#---dataset
class FROGDataset:
    """
    FROG Dataset.
    """
    def __init__(self, batch_size=4, dataset_path='/content/drive/MyDrive/MachineLearningProject2020/FROG_uniform_pulse_duration'):
        self.batch_size = batch_size
        self.dataset_path = dataset_path
        self.train_dataset = self.get_train_numpy()
        self.x_mean, self.x_std = self.compute_train_statistics()
        self.transform = self.get_transforms()
        self.train_loader, self.val_loader = self.get_dataloaders()
        

    def get_train_numpy(self):
        train_dataset = train_data
        return train_dataset

    def compute_train_statistics(self):
        # TODO (part a): compute per-channel mean and std with respect to self.train_dataset
        print(self.train_dataset.shape) # should be a (8000 x 64 x 64) array
        x_mean = np.array(np.mean(self.train_dataset))  # per-channel mean // only one channel of values
        x_std = np.array(np.std(self.train_dataset))  # per-channel std
        return x_mean, x_std

    def get_transforms(self):
        # TODO (part a): fill in the data transforms
        transform_list = [
            transforms.ToTensor(),
            transforms.Normalize(self.x_mean,self.x_std)
        ]
        transform = transforms.Compose(transform_list)
        return transform

    def get_dataloaders(self):
        train_dataset = train_data
        train_img = np.zeros((len(train_dataset),3,64,64)) # 3 channels for CNN
        train_label = train_labels # using the one-hot labels (10x1 numpy array)
        for i in range(len(train_dataset)):
            image = train_dataset[i,:,:]
            image_temp = np.array([image,image,image])
            train_img[i] = image_temp
        train_img = torch.from_numpy(train_img).float()
        train_label = torch.from_numpy(train_label).float()
        train_set = torch.utils.data.TensorDataset(train_img,train_label)
        train_loader = torch.utils.data.DataLoader(train_set, batch_size=self.batch_size, shuffle=True)

        # validation set
        val_dataset = val_data
        val_img = np.zeros((len(val_dataset),3,64,64))
        val_label = val_labels # using the one-hot labels (10x1 numpy array)
        for i in range(len(val_dataset)):
            image = val_dataset[i,:,:]
            image_temp = np.array([image,image,image])
            val_img[i] = image_temp
        val_img = torch.from_numpy(val_img).float()
        val_label = torch.from_numpy(val_label).float()
        val_set = torch.utils.data.TensorDataset(val_img,val_label)
        val_loader = torch.utils.data.DataLoader(val_set, batch_size=self.batch_size, shuffle=False)

        return train_loader, val_loader

class CNN(nn.Module):
    """
    Convolutional Neural Network.
    """
    def __init__(self):
        super().__init__()

        # TODO (part b): define layers
        self.mr1_11 = nn.Conv2d(3, 3, 11, stride=2, padding=37)  # multires layer 1 -> 11x11 filter
        self.mr1_7 = nn.Conv2d(3, 3, 7, stride=2, padding=35)  # multires layer 1 -> 7x7 filter
        self.mr1_5 = nn.Conv2d(3, 3, 5, stride=2, padding=34)  # multires layer 1 -> 5x5 filter
        self.mr1_3 = nn.Conv2d(3, 3, 3, stride=2, padding=33)  # multires layer 1 -> 3x3 filter
        self.conv1 = nn.Conv2d(12, 24, 5, stride=2, padding=2)  # convolutional layer 1
        self.mr2_11 = nn.Conv2d(24, 24, 11, stride=2, padding=21)  # multires layer 2 -> 11x11 filter
        self.mr2_7 = nn.Conv2d(24, 24, 7, stride=2, padding=19)  # multires layer 2 -> 7x7 filter
        self.mr2_5 = nn.Conv2d(24, 24, 5, stride=2, padding=18)  # multires layer 2 -> 5x5 filter
        self.mr2_3 = nn.Conv2d(24, 24, 3, stride=2, padding=17)  # multires layer 2 -> 3x3 filter
        self.conv2 = nn.Conv2d(96, 192, 5, stride=2, padding=2)  # convolutional layer 2
        self.mr3_11 = nn.Conv2d(192, 192, 11, stride=2, padding=13)  # multires layer 3 -> 11x11 filter
        self.mr3_7 = nn.Conv2d(192, 192, 7, stride=2, padding=11)  # multires layer 3 -> 7x7 filter
        self.mr3_5 = nn.Conv2d(192, 192, 5, stride=2, padding=10)  # multires layer 3 -> 5x5 filter
        self.mr3_3 = nn.Conv2d(192, 192, 3, stride=2, padding=9)  # multires layer 3 -> 3x3 filter
        self.conv3 = nn.Conv2d(768, 1536, 5, stride=2, padding=2)  # convolutional layer 3
        self.fc1 = nn.Linear(1536 * 8 * 8, 512)  # fully connected layer 1
        self.fc2 = nn.Linear(512, 1)  # fully connected layer 2 (output layer for pulse width prediction)

        self.init_weights()

    def init_weights(self):
        for conv in [self.conv1, self.conv2, self.conv3]:
            C_in = conv.weight.size(1)
            nn.init.normal_(conv.weight, 0.0, 1 / math.sqrt(5 * 5 * C_in))
            nn.init.constant_(conv.bias, 0.0)

        for mr11 in [self.mr1_11, self.mr2_11, self.mr3_11]:
            C_in = mr11.weight.size(1)
            nn.init.normal_(mr11.weight, 0.0, 1 / math.sqrt(11 * 11 * C_in))
            nn.init.constant_(mr11.bias, 0.0)

        for mr7 in [self.mr1_7, self.mr2_7, self.mr3_7]:
            C_in = mr7.weight.size(1)
            nn.init.normal_(mr7.weight, 0.0, 1 / math.sqrt(7 * 7 * C_in))
            nn.init.constant_(mr7.bias, 0.0)

        for mr5 in [self.mr1_5, self.mr2_5, self.mr3_5]:
            C_in = mr5.weight.size(1)
            nn.init.normal_(mr5.weight, 0.0, 1 / math.sqrt(5 * 5 * C_in))
            nn.init.constant_(mr5.bias, 0.0)       

        for mr3 in [self.mr1_3, self.mr2_3, self.mr3_3]:
            C_in = mr3.weight.size(1)
            nn.init.normal_(mr3.weight, 0.0, 1 / math.sqrt(3 * 3 * C_in))
            nn.init.constant_(mr3.bias, 0.0)

        for fc in [self.fc1, self.fc2]:
            F_in = fc.weight.size(1)
            nn.init.normal_(fc.weight, 0.0, 1 / math.sqrt(F_in))
            nn.init.constant_(fc.bias, 0.0)


    def forward(self, x):
        N, C, H, W = x.shape
        z1 = self.mr1_11(x)
        z2 = self.mr1_7(x)
        z3 = self.mr1_5(x)
        z4 = self.mr1_3(x)
        z = F.relu(torch.cat((z1,z2,z3,z4),dim=1))

        z = F.relu(self.conv1(z))
        z5 = self.mr2_11(z)
        z6 = self.mr2_7(z)
        z7 = self.mr2_5(z)
        z8 = self.mr2_3(z)
        z = F.relu(torch.cat((z5,z6,z7,z8),dim=1))

        z = F.relu(self.conv2(z))
        z9 = self.mr3_11(z)
        z10 = self.mr3_7(z)
        z11 = self.mr3_5(z)
        z12 = self.mr3_3(z)
        z = F.relu(torch.cat((z9,z10,z11,z12),dim=1))

        z = F.relu(self.conv3(z))
        z = z.view(-1, 1536 * 8 * 8)
        z = F.relu(self.fc1(z))
        z = self.fc2(z)

        return z


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


if __name__ == '__main__':
    # import FROGDataset
    net = CNN()
    net = net.to(device)
    print(net)
    print('Number of CNN parameters: {}'.format(count_parameters(net)))

#---train
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

def predictions(logits):
    """
    Compute the predictions from the model.
    Inputs:
        - logits: output of our model based on some input, tensor with shape=(batch_size, num_features)
    Returns:
        - pred: predictions of our model, tensor with shape=(batch_size,num_features)
    """
    return logits


def accuracy(y_true, y_pred):
    """
    Compute the accuracy given true and predicted labels.
    Inputs:
        - y_true: true labels, tensor with shape=(num_examples)
        - y_pred: predicted labels, tensor with shape=(num_examples)
    Returns:
        - acc: accuracy, float
    """

    return 0.0

def _train_epoch(train_loader, model, criterion, optimizer):
    """
    Train the model for one iteration through the train set.
    """
    
    for i, (X, y) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(X.to(device))
        loss = criterion(output, y.to(device))
        loss.backward()
        optimizer.step()

def train(config, val_data, model):
    criterion = torch.nn.L1Loss()
    learning_rate = config['learning_rate']
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
    print('Loading model...')
    force = config['ckpt_force'] if 'ckpt_force' in config else False
    model, start_epoch, stats = restore_checkpoint(model, config['ckpt_path'], force=force)
    model.eval()
    
    pred = model(val_data)

    return pred
    print('Finished Training')
if __name__ == '__main__':
    # define config parameters for training
    config = {
        'batch_size': 4,
        'ckpt_path': '/content/drive/MyDrive/MultiRes',  # directory to save our model checkpoints
        'num_epoch': 40,                 # number of epochs for training
        'learning_rate': 1e-5,           # learning rate
        'plot_name': 'Multi-Res'        # plot name
    }
    model = CNN()
    df = pd.read_pickle('/content/drive/MyDrive/Test_domain_generalization/Pyret_Labels_for_CNN_5_15_duration_1000.pkl')
    np.random.seed(5)
    idx =np.random.randint(1000, size=(600))
    val_data = np.tile(np.expand_dims(df.iloc[0,1][idx,:,:],axis = 1),[1,3,1,1])
    val_label = np.array(df.iloc[0,2])[idx]
    val_label = torch.tensor(val_label)
    val_data = torch.tensor(val_data).float()
    val_data = val_data.to(device)
    val_labelll = np.array(df.iloc[0,0])[idx,:]
    len_pred = train(config,val_data, model.to(device))
    query_idx = len_pred.cpu().detach().numpy().astype(int) - 5


In [ ]:
"""
Save the weights and biases of the FC layers queried based on the MultiRes predictions
"""
    fc_layers = pd.read_pickle('/content/drive/MyDrive/Test_domain_generalization/fc_layers.pkl')
    fc_layers1_bias = fc_layers.loc[0, 'fc_layers1_bias']
    fc_layers2_bias = fc_layers.loc[0, 'fc_layers2_bias']
    fc_layers3_bias = fc_layers.loc[0, 'fc_layers3_bias']
    fc_layers1_weight = fc_layers.loc[0, 'fc_layers1_weight']
    fc_layers2_weight = fc_layers.loc[0, 'fc_layers2_weight']
    fc_layers3_weight = fc_layers.loc[0, 'fc_layers3_weight']
    fc_layers1_b = np.squeeze(fc_layers1_bias[query_idx,:]).cpu().detach().numpy()
    fc_layers2_b = np.squeeze(fc_layers2_bias[query_idx,:]).cpu().detach().numpy()
    fc_layers3_b = np.squeeze(fc_layers3_bias[query_idx,:]).cpu().detach().numpy()
    fc_layers1_w = np.squeeze(fc_layers1_weight[query_idx,:,:]).cpu().detach().numpy()
    fc_layers2_w = np.squeeze(fc_layers2_weight[query_idx,:, :]).cpu().detach().numpy()
    fc_layers3_w = np.squeeze(fc_layers3_weight[query_idx,:, :]).cpu().detach().numpy()
    fc_layers1_w = np.swapaxes(fc_layers1_w,1,2)
    fc_layers2_w = np.swapaxes(fc_layers2_w,1,2)
    fc_layers3_w = np.swapaxes(fc_layers3_w,1,2)

In [ ]:
"""
Evaluate the loss for our model
"""
def load_pretrained(dict_path):
    Resnet18 = models.resnet18(pretrained=False)
    if (dict_path != None):
        Resnet18.load_state_dict(torch.load(dict_path))
    for param in Resnet18.parameters():
        param.requires_grad = True
    n_inputs = Resnet18.fc.in_features
    Resnet18.fc = torch.nn.Linear(512,1000)
    return Resnet18

def train(config, dataset, model):
    print('Loading model...') 
    force = config['ckpt_force'] if 'ckpt_force' in config else False
    model, start_epoch, stats = restore_checkpoint(model, config['ckpt_path'], force=force)
    model = nn.Sequential(*list(model.children())[:-1])
    print('Finished Training')
    return model

if __name__ == '__main__':
    config = {
        'batch_size': 4,
        'ckpt_path': '/content/drive/MyDrive/checkpointsResNet4e-4_120_24_duration_5_15_2fc_long/transfer',
        'plot_name': 'TransferCPU_4_4e-3_50EP',
        'dict_path': '/content/drive/MyDrive/ResNet18.pt',

    }
    model = load_pretrained(config['dict_path'])
    lin = model.fc
    new_lin = nn.Sequential(
        lin,
        nn.LeakyReLU(0.2),
        nn.Linear(lin.out_features, 1000),
        nn.LeakyReLU(0.2),
        nn.Linear(1000, 128)
    )
    model.fc = new_lin
    model = model.cuda()
    model = train(config, val_data, model)  
    val_pr = model(val_data)
    Leaky = nn.LeakyReLU(0.2)
    val_pr= val_pr.cpu().detach().numpy()
    val_pr = np.squeeze(val_pr)
    val_pr = np.expand_dims(val_pr,1)
    fc_layers1_w = fc_layers1_w
    inter1 = torch.tensor(np.squeeze(np.matmul(val_pr,fc_layers1_w))+ fc_layers1_b)
    inter1 = Leaky(inter1).cpu().detach().numpy()

    inter2 = torch.tensor(np.squeeze(np.matmul(np.expand_dims(inter1,1),fc_layers2_w))+ fc_layers2_b)
    inter2 = Leaky(inter2).cpu().detach().numpy()
    inter3 = np.squeeze(np.matmul(np.expand_dims(inter2,1),fc_layers3_w))+ fc_layers3_b
    plt.plot(inter3[2,:])
    plt.plot(val_labelll[2,:])
predd = torch.tensor(inter3)
vall = torch.tensor(val_labelll)
L1 = nn.L1Loss()
print('The loss of our model is: ',L1(predd,vall))

In [ ]:
"""
Evaluate the loss for the ResNet base model
"""
def train(config, dataset, model):

    print('Loading model...') 
    force = config['ckpt_force'] if 'ckpt_force' in config else False
    model, start_epoch, stats = restore_checkpoint(model, config['ckpt_path'], force=force)
    print('Finished Training')
    return model

if __name__ == '__main__':
    config = {
        'batch_size': 4,
        'ckpt_path': '/content/drive/MyDrive/checkpointsResNet4e-4_120_24_duration_5_15_2fc_long/transfer',
        'plot_name': 'TransferCPU_4_4e-3_50EP',
        'dict_path': '/content/drive/MyDrive/ResNet18.pt',

    }
    model = load_pretrained(config['dict_path'])
    lin = model.fc
    new_lin = nn.Sequential(
        lin,
        nn.LeakyReLU(0.2),
        nn.Linear(lin.out_features, 1000),
        nn.LeakyReLU(0.2),
        nn.Linear(1000, 128)
    )
    model.fc = new_lin
    model = model.cuda()
    model = train(config, val_data, model)  
    pred_base = model(val_data)
    print("The loss of ResNet base model is: ", L1(pred_base,vall.to('cuda')))